# **Важно!** 

Домашнее задание состоит из нескольких задач, которые вам нужно решить.
*   Баллы выставляются по принципу выполнено/невыполнено.
*   За каждую выполненую задачу вы получаете баллы (количество баллов за задание указано в скобках).

**Инструкция выполнения:** Выполните задания в этом же ноутбуке (места под решения **КАЖДОЙ** задачи обозначены как **#НАЧАЛО ВАШЕГО РЕШЕНИЯ** и **#КОНЕЦ ВАШЕГО РЕШЕНИЯ**)

**Как отправить задание на проверку:** Вам необходимо сохранить ваше решение в данном блокноте и отправить итоговый **файл .IPYNB** в личном сообщении Telegram.

# **Прежде чем проверять задания:**

1. Перезапустите **ядро (restart the kernel)**: в меню, выбрать **Ядро (Kernel)**
→ **Перезапустить (Restart)**
2. Затем **Выполнить** **все ячейки (run all cells)**: в меню, выбрать **Ячейка (Cell)**
→ **Запустить все (Run All)**.

---

## Введение в Parameter Efficient Fine-Tuning (PEFT)

В данном задании основное внимание уделяется изучению и применению современных методов **Parameter Efficient Fine-Tuning (PEFT)**. Эти подходы позволяют эффективно дообучать большие языковые модели, используя лишь небольшую часть параметров, что критически важно при работе с ограниченными вычислительными ресурсами.

### Обязательные PEFT методы для изучения:

1. **LoRA (Low-Rank Adaptation)**
   - Разложение весовых матриц на произведение матриц низкого ранга
   - Ключевые гиперпараметры: `r` (rank), `alpha`, `dropout`, `target_modules`

2. **QLoRA (Quantized LoRA)** 
   - Комбинация 4-bit квантизации (NF4) с LoRA
   - Значительно снижает потребление GPU памяти

3. **AdaLoRA (Adaptive LoRA)**
   - Динамическое изменение ранга во время обучения
   - Автоматическая оптимизация распределения параметров

### Дополнительные методы (по выбору):
- **IA³ (Infused Adapter by Inhibiting and Amplifying Inner Activations)**
- **Prefix Tuning / P-Tuning v2**
- **Prompt Tuning**

### Методология сравнения PEFT подходов:

Для каждого метода необходимо измерить и сравнить:

#### Эффективность ресурсов:
- **Количество обучаемых параметров** (в % от общего числа параметров модели)
- **Потребление GPU памяти** (в GB во время обучения и инференса)
- **Время обучения** (сек/эпоху)
- **Скорость инференса** (токенов/секунду)

#### Качество результатов:
- **Основные метрики** в зависимости от задачи (ROUGE для суммаризации, BLEU для перевода)
- **Стабильность обучения** (сходимость функции потерь)
- **Качественный анализ** выходных текстов

#### Требования к отчету:
1. **Сравнительная таблица** всех протестированных методов
2. **Графики Pareto-frontier**: эффективность vs качество
3. **Обоснованные рекомендации** по выбору метода для различных сценариев
4. **Анализ компромиссов** между точностью и эффективностью

---

### Задание 1: Parameter Efficient Fine-Tuning (PEFT) моделей для суммаризации текстов

Цель задания — научиться применять современные методы параметрически-эффективного дообучения (PEFT) для настройки больших языковых моделей на задачу суммаризации текстов. Вы реализуете различные PEFT подходы: LoRA, QLoRA из библиотеки PEFT.

#### Задачи:

1. **Выбор датасета**:
   - Загрузите датасет для задачи суммаризации, например, датасет `CNN/DailyMail`, содержащий новостные статьи и их краткие содержания (референсы). Используйте библиотеку `datasets` для загрузки данных.
   - Обратите внимание на то, что вы можете использовать и другие подходящие датасеты для суммаризации (например, XSum).

2. **Предобработка данных**:
   - **Разделите** данные на обучающую и тестовую выборки.
   - **Очистите текст** от лишних символов, специальных токенов и пробелов.
   - **Подготовьте данные** в формате, подходящем для выбранной модели:
     - Для GPT-2 вам нужно будет подать текст целиком (входной текст + референсное суммирование в одном формате).
     - Для T5 модель требует форматировать входные данные в виде `summarize: <текст>` для текстов, которые нужно суммировать.

3. **Создание модели**:
   - **GPT-2**:
     - Импортируйте предобученную модель `GPT2LMHeadModel` из библиотеки Hugging Face.
     - Модель GPT-2 изначально не предобучена для задачи суммаризации, поэтому требуется её дообучение на подходящих данных.
     - Поддержите правильное управление длиной сгенерированного текста при инференсе, чтобы обеспечить краткость суммаризаций.
   
   - **T5**:
     - Для T5 используйте модель `T5ForConditionalGeneration`. T5 уже предобучена на множестве задач, включая суммаризацию, поэтому она лучше подходит для данной задачи.
     - В отличие от GPT-2, модель T5 обучена на задаче, где входной текст — это задание (например, "summarize:") + текст для обработки, а выход — краткое содержание. Это нужно учесть при подготовке данных.
     - Импортируйте модель из библиотеки Hugging Face и настройте её для использования на задаче суммаризации.

4. **Настройка параметров обучения**:
   - Настройте параметры обучения для обеих моделей:
     - Количество эпох.
     - Размер батча.
     - Скорость обучения.
     - Выберите оптимизатор (например, AdamW).
   - Для T5 используйте кросс-энтропийную функцию потерь (`CrossEntropyLoss`), так как это задача генерации текста с "условной вероятностью". Для GPT-2 используйте ту же функцию с учётом автогрегрессивного генерационного процесса.

5. **PEFT Fine-tuning модели** (основная часть задания):
   - **Обязательно** реализуйте дообучение с использованием различных PEFT методов:
     - **LoRA (Low-Rank Adaptation)**: Настройте параметры rank (r), alpha, dropout
     - **QLoRA (Quantized LoRA)**: Используйте 4-bit квантизацию с LoRA
     - **AdaLoRA**: Адаптивное изменение ранга во время обучения
     - **Дополнительно**: попробуйте IA³ (Infused Adapter by Inhibiting and Amplifying Inner Activations) или Prompt Tuning
   - Для каждого PEFT метода:
     - Настройте специфические гиперпараметры (rank, alpha, target_modules)
     - Измерьте количество обучаемых параметров
     - Зафиксируйте время обучения и потребление памяти
   - Используйте класс `Trainer` или `SFTTrainer` из библиотеки `trl` для процесса дообучения
   - Сравните результаты full fine-tuning с PEFT подходами (опционально)

6. **Инференс**:
   - Используйте обе модели для генерации кратких содержаний на тестовой выборке.
   - Подготовьте несколько примеров суммаризаций и выведите результаты для каждой модели.
   - Для инференса используйте разные стратегии декодирования:
     - **Greedy decoding** (жадный поиск).
     - **Beam search** (поиск по нескольким лучам).
     - **Sampling** (стохастическая генерация с использованием вероятностей).
   - Сравните результаты, чтобы понять, как разные стратегии влияют на качество суммаризаций.

7. **Сравнительная оценка PEFT методов**:
   - Создайте сравнительную таблицу для всех протестированных методов:
     - Количество обучаемых параметров (в % от общего числа параметров модели)
     - Время обучения на эпоху
     - Потребление GPU памяти
     - Качество суммаризации по метрикам ROUGE и BLEU
   - Оцените качество суммаризаций с использованием метрик:
     - **ROUGE-1, ROUGE-2, ROUGE-L** — для оценки точности и полноты суммаризаций
     - **BLEU** — для оценки схожести с референсным текстом
   - Проанализируйте trade-off между эффективностью обучения и качеством результата

#### Ожидаемые результаты:
- **Код с реализацией различных PEFT методов** для выбранной модели (GPT-2 или T5)
- **Сравнительный анализ** всех протестированных PEFT подходов с детальными метриками эффективности
- **Отчет** с обоснованием выбора оптимального PEFT метода для задачи суммаризации
- **Примеры суммаризаций** для каждого PEFT метода с качественным анализом различий
- **Рекомендации** по выбору PEFT подхода в зависимости от ограничений по ресурсам

#### Рекомендуемые ресурсы:
- **[PEFT Documentation](https://huggingface.co/docs/peft/index)** - основная документация библиотеки PEFT
- **[LoRA Developer Guide](https://huggingface.co/docs/peft/developer_guides/lora)** - детальное руководство по LoRA
- **[QLoRA Implementation](https://huggingface.co/docs/peft/developer_guides/quantization)** - квантизация и QLoRA
- [Документация Hugging Face Transformers](https://huggingface.co/docs/transformers/index)
- [Документация Hugging Face Datasets](https://huggingface.co/docs/datasets/index)
- **[PEFT Examples](https://github.com/huggingface/peft/tree/main/examples)** - примеры использования различных PEFT методов
- **[TRL SFTTrainer](https://huggingface.co/docs/trl/sft_trainer)** - для supervised fine-tuning

#### Критерии оценки:
- **Корректность реализации PEFT методов** (30%) - правильная настройка и применение различных PEFT подходов
- **Полнота сравнительного анализа** (25%) - детальное сравнение методов по всем указанным метрикам
- **Качество сгенерированных суммаризаций** (20%) - оценка по ROUGE/BLEU метрикам
- **Глубина анализа и выводов** (15%) - обоснованные рекомендации по выбору PEFT метода
- **Четкость и структурированность отчета** (10%) - качество документации процесса и результатов

In [ ]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
raise NotImplementedError() # удалить эту строку в процессе решения
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

### Задание 2: Применение различных PEFT методов для задачи машинного перевода

**Цель задания**: Сравнить эффективность различных Parameter Efficient Fine-Tuning (PEFT) подходов для задачи автоматического перевода с английского на русский. В рамках задания необходимо реализовать и сравнить минимум 3 различных PEFT метода, проанализировать их влияние на качество перевода и эффективность обучения.

#### Задачи:

1. **Выбор датасета**:
   - Загрузите параллельный датасет для перевода, например, [Opus Books](https://huggingface.co/datasets/Helsinki-NLP/opus_books/viewer/en-ru), который содержит тексты на английском языке и их переводы на русский.
   - Используйте библиотеку `datasets` для загрузки и обработки данных. Убедитесь, что данные содержат параллельные тексты для обучения модели переводу.

2. **Предобработка данных**:
   - **Разделите данные** на обучающую и тестовую выборки (например, 80% для обучения и 20% для тестирования).
   - **Очистите текст**, удалив лишние пробелы и специальные символы, которые могут повлиять на обучение модели.
   - **Подготовьте данные** в формате, подходящем для выбранной модели:
     - Для GPT-2 данные должны быть в виде последовательности токенов, где исходный текст и перевод разделены специальными символами (например, `<|startoftext|>` для начала текста и `<|endoftext|>` для его конца).
     - Для T5 данные подаются в формате задачи перевода: входной текст начинается с задания `"translate English to Russian: <текст на английском>"`, а на выходе модель должна сгенерировать перевод.

3. **Создание модели**:
   - **GPT-2**:
     - Импортируйте предобученную модель `GPT2LMHeadModel` из библиотеки Hugging Face.
     - GPT-2 изначально не обучена для задачи перевода, поэтому нужно будет использовать специальную подготовку данных и дообучение на параллельных текстах.
     - Убедитесь, что модель настроена для генерации текста, ограничивая длину вывода для перевода.
   
   - **T5**:
     - Импортируйте модель `T5ForConditionalGeneration`, которая предобучена на множестве задач, включая перевод. T5 — это модель с условной генерацией, которая использует специальную задачу (`task`) для перевода.
     - Подготовьте модель для выполнения задачи перевода с английского на русский, используя предобученные веса и формат входных данных.

4. **Настройка параметров обучения**:
   - Настройте параметры обучения для обеих моделей:
     - Количество эпох (например, 3-5 эпох).
     - Размер батча (например, 16 или 32).
     - Скорость обучения (рекомендуется начать с 5e-5 и адаптировать в зависимости от потерь на валидации).
   - Используйте подходящие оптимизаторы, такие как AdamW, и функцию потерь для задачи перевода:
     - Для T5 подойдёт стандартная кросс-энтропийная функция потерь.
     - Для GPT-2 используйте ту же функцию с учётом последовательной генерации текста (автогрегрессии).

5. **Сравнительное исследование PEFT методов** (ключевая часть задания):
   - **Обязательно** реализуйте и сравните следующие PEFT подходы:
     - **LoRA**: Настройте различные значения rank (4, 8, 16, 32), alpha (16, 32, 64)
     - **QLoRA**: Комбинация 4-bit квантизации с LoRA для экономии памяти
     - **AdaLoRA**: Адаптивное изменение ранга с бюджетом параметров
     - **Дополнительные методы** (на выбор): IA³, Prefix Tuning, P-Tuning v2, или (IA)³
   - Для каждого PEFT метода проведите **grid search** по ключевым гиперпараметрам
   - **Baseline**: обязательно сравните с полным fine-tuning (если позволяют ресурсы)
   - Зафиксируйте для каждого эксперимента:
     - Процент обучаемых параметров от общего числа
     - Потребление GPU памяти (в GB)
     - Время обучения на эпоху
     - Скорость инференса (токенов/сек)
   - Используйте `SFTTrainer` или `Trainer` для стабильного процесса обучения

6. **Инференс**:
   - Используйте дообученную модель для перевода текстов с английского на русский на тестовой выборке.
   - Подготовьте несколько примеров для перевода и выведите результаты:
     - Для GPT-2 используйте автогрегрессивное декодирование текста.
     - Для T5 применяйте стандартные стратегии декодирования (например, greedy decoding или beam search).
   
7. **Комплексная оценка и анализ PEFT методов**:
   - **Автоматические метрики**:
     - **BLEU** (corpus-level и sentence-level) — основная метрика для машинного перевода
     - **chrF** — character-level F-score для более точной оценки морфологически богатых языков
     - **COMET** — нейронная метрика качества перевода (если позволяют ресурсы)
     - **ROUGE** — для дополнительной оценки похожести
   - **Создайте детальную сравнительную таблицу**:
     - Эффективность (% параметров, время, память) vs Качество (BLEU, chrF)
     - Pareto-frontier анализ: какие методы обеспечивают лучший trade-off
   - **Качественный анализ**: 
     - Проанализируйте примеры переводов от каждого PEFT метода
     - Определите типы ошибок, характерные для каждого подхода
     - Оцените стабильность качества на различных типах текстов

#### Ожидаемые результаты:
- **Исследовательский код** с реализацией всех протестированных PEFT методов
- **Научно-обоснованный отчет** с детальным сравнением эффективности PEFT подходов
- **Визуализации**: графики Pareto-frontier, сравнительные диаграммы по метрикам
- **Практические рекомендации**: когда использовать каждый PEFT метод в зависимости от ограничений
- **Воспроизводимые результаты**: четкие инструкции по повторению экспериментов

#### Рекомендуемые ресурсы:
- **[PEFT Documentation](https://huggingface.co/docs/peft/index)** - полная документация библиотеки PEFT
- **[LoRA Paper & Implementation](https://arxiv.org/abs/2106.09685)** - оригинальная статья LoRA
- **[QLoRA Paper](https://arxiv.org/abs/2305.14314)** - квантизованный LoRA подход
- **[AdaLoRA Paper](https://arxiv.org/abs/2303.10512)** - адаптивный LoRA
- **[PEFT Task Guides](https://huggingface.co/docs/peft/task_guides/translation)** - гайды по применению PEFT для разных задач
- [Документация Hugging Face Transformers](https://huggingface.co/docs/transformers/index)
- [Документация Hugging Face Datasets](https://huggingface.co/docs/datasets/index)
- **[BitsAndBytes](https://huggingface.co/docs/bitsandbytes/index)** - для квантизации в QLoRA
- **[TRL Library](https://huggingface.co/docs/trl/index)** - для эффективного обучения

#### Критерии оценки:
- **Корректность реализации PEFT методов** (35%) - правильная настройка минимум 3 различных PEFT подходов
- **Качество сравнительного анализа** (25%) - детальное сравнение по всем метрикам эффективности и качества
- **Научная обоснованность выводов** (20%) - аргументированные рекомендации по выбору методов
- **Воспроизводимость результатов** (10%) - четкие инструкции и фиксированные семена
- **Качество перевода** (10%) - достижение конкурентоспособных результатов по BLEU/chrF метрикам

In [ ]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
raise NotImplementedError() # удалить эту строку в процессе решения
# КОНЕЦ ВАШЕГО РЕШЕНИЯ